Источник: https://github.com/PacktPublishing/Hands-On-Image-Processing-with-Python/tree/master

In [357]:
# !pip install -U scikit-image

In [358]:
import os
from PIL import Image, ImageFilter
import numpy as np
from numpy import asarray
import shutil
from scipy.ndimage import gaussian_filter
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage

In [359]:
# constants

# 1
P = 256

# 4
K = 5

In [360]:
# functions
def remove(path):
    if os.path.exists(path):  
        if os.path.isfile(path) or os.path.islink(path):
            os.unlink(path)
        else:
            shutil.rmtree(path)

def contrast_stretching(img: np.array) -> np.array:
    return ((img - img.min()) / (img.max() - img.min()) * (P-1)).astype(np.uint8)

In [361]:
for i in range(1, 8):
    remove(f"files/{i}")
for i in range(1, 8):
    os.mkdir(f"files/{i}")


img = Image.open("files/image.jpg")

### 1. Загрузка и визуализация изображения
#### a) Числовые характеристики изображения

In [362]:
print(f"Размерность изображения: {img.size[0]}x{img.size[1]}")
print(f"Число световых каналов: {len(img.getbands())}")
print(f"Яркостное разрешение: {len(img.getbands()) * 8} bits")


Размерность изображения: 1600x1060
Число световых каналов: 3
Яркостное разрешение: 24 bits


#### б) Если изображение содержит много цветовых каналов, то привести его к одноканальному (в градациях серого) в диапазоне [0; P], P = 255 (яркостное разрешение – 1 байт) 

In [363]:
gray = img.convert('L')
gray.save('files/greyscale.jpg')

img_array = asarray(gray)

### 2. Градиентные операторы

#### a) Рассчитать градиентные изображения в вертикальном и горизонтальном направлениях, а также изображения модулей и углов градиентов, используя оператор Робертса
Источник: https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D0%B5%D0%BA%D1%80%D1%91%D1%81%D1%82%D0%BD%D1%8B%D0%B9_%D0%BE%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80_%D0%A0%D0%BE%D0%B1%D0%B5%D1%80%D1%82%D1%81%D0%B0

Источник: https://habr.com/ru/articles/114452/

In [364]:
img_array.shape

(1060, 1600)

In [365]:
roberts_cross_v = np.array( [[1, 0 ], 
                             [0,-1 ]] ) 
  
roberts_cross_h = np.array( [[ 0, 1 ], 
                             [ -1, 0 ]] )

g_vertical = ndimage.convolve(img_array, roberts_cross_v)
Image.fromarray(g_vertical).save('files/1/vertical.jpg')

g_horizontal = ndimage.convolve(img_array, roberts_cross_h) 
Image.fromarray(g_horizontal).save('files/1/horizontal.jpg')

g_module = np.sqrt( np.square(g_horizontal) + np.square(g_vertical)).astype(np.int8)
Image.fromarray(g_module, "L").save('files/1/module.jpg')

g_angle = np.arctan(g_vertical/g_horizontal).astype(np.int8)
Image.fromarray(g_angle * (P - 1), "L").save('files/1/angle.jpg')

/tmp/ipykernel_404/897329376.py:16: RuntimeWarning: divide by zero encountered in divide
  g_angle = np.arctan(g_vertical/g_horizontal).astype(np.int8)
/tmp/ipykernel_404/897329376.py:16: RuntimeWarning: invalid value encountered in divide
  g_angle = np.arctan(g_vertical/g_horizontal).astype(np.int8)
/tmp/ipykernel_404/897329376.py:16: RuntimeWarning: invalid value encountered in cast
  g_angle = np.arctan(g_vertical/g_horizontal).astype(np.int8)


In [366]:
g_angle.min()

0

### 3) Применить к изображению фильтр Гаусса с различными значениями среднеквадратического отклонения sigma:

In [367]:
for sigma in (0.5, 5, 10, 20):
    image = gray.filter(ImageFilter.GaussianBlur(sigma))
    image.save(f"files/3/gaussian_filter_sigma_{sigma}.jpg")

### 5) Применить к изображению фильтр лапласиана гауссиан (LoG) с различными размером окна k и значениями среднеквадратического отклонения sigma:
Источник: https://math.stackexchange.com/questions/2445994/discrete-laplacian-of-gaussian-log

Источник: https://homepages.inf.ed.ac.uk/rbf/HIPR2/zeros.htm

#### а) Ядро k = 5, sigma = 0.2

In [368]:
# Проверка для смены типа
np.array_equal(img_array, img_array.astype(np.int16))

True

In [369]:
def LOG(sigma, x, y):
    laplace = -1/(np.pi*sigma**4)*(1-(x**2+y**2)/(2*sigma**2))*np.exp(-(x**2+y**2)/(2*sigma**2))
    return laplace

def LOG_discrete(sigma, n):
    l = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            l[i,j] = LOG(sigma, (i-(n-1)/2),(j-(n-1)/2))
    return l

In [370]:
def LOG_process(k, sigma):
    results = ndimage.convolve(img_array.astype(np.int16), LOG_discrete(sigma, k))

    binary_results = results.copy()
    binary_results[binary_results > 0] = 1
    binary_results[binary_results < 0] = -1

    zero_crossing_results = results.copy()
    zero_crossing_results[zero_crossing_results > 0] = -1
    zero_crossing_results[zero_crossing_results < 0] = -1
    zero_crossing_results[zero_crossing_results == 0] = 0
    return results, binary_results, zero_crossing_results

In [371]:
k, sigma = 5, 0.2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

/tmp/ipykernel_404/2934881476.py:10: RuntimeWarning: overflow encountered in scalar subtract
  return ((img - img.min()) / (img.max() - img.min()) * (P-1)).astype(np.uint8)


In [372]:
k, sigma = 10, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [373]:
k, sigma = 30, 2
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

In [374]:
k, sigma = 30, 10
results, binary_results, zero_crossing_results = LOG_process(k, sigma)

Image.fromarray(contrast_stretching(results)).save(f"files/5/log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(binary_results)).save(f"files/5/binary_log_k_{k}_sigma_{sigma}.jpg")
Image.fromarray(contrast_stretching(zero_crossing_results)).save(f"files/5/zero_crossing_log_k_{k}_sigma_{sigma}.jpg")

### 6. Применить к изображению фильтр разности гауссиан (DoG) с различными параметрами sigma1 и alpha = sigma1 / sigma2
##### Источник: https://gist.github.com/leonidk/8798fdbf38db120b8536d25ea2f8c3b4

In [375]:
def DOG(img, sigma1, alpha):
    sigma2 = sigma1 / alpha
    s1 = gaussian_filter(img, sigma1)
    s2 = gaussian_filter(img, sigma2)

    dog = s1 - s2
    return dog

In [376]:
Image.fromarray(DOG(img_array, 2, 1.6)).save(f"files/6/dog_sigma1_{2}_alpha_{1.6}.jpg")
Image.fromarray(DOG(img_array, 5, 1.6)).save(f"files/6/dog_sigma1_{5}_sigma_{1.6}.jpg")
Image.fromarray(DOG(img_array, 2, 5)).save(f"files/6/dog_sigma1_{2}_sigma_{5}.jpg")
Image.fromarray(DOG(img_array, 2, 10)).save(f"files/6/dog_sigma1_{2}_sigma_{10}.jpg")

### 7. Выделить границы на изображении

#### а) Детектор Марра-Хилдрета

##### https://github.com/adl1995/edge-detectors/blob/master/marr-hildreth-edge.py